# Importings

- consider imputing some of the tests
- take the LGBM or Catboost Performance o
- try different values for clipping

In [1]:
import pandas as pd                                    
import numpy as np                                      
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

class CFG:
  data_folder ="/workspaces/khwarizm/data/"

def reading_data(path: str) -> pd.DataFrame:
  train = pd.read_csv(path + 'Train.csv')
  test = pd.read_csv(path + 'Test.csv')
  return train,test

train,test = reading_data(CFG.data_folder)

groups = train['ID']
test_id = test['ID_Zindi']
train_date = train['Date']
test_date = test['Date']
test_id = test['ID_Zindi']
pd.options.display.max_columns = 200
#train = train.dropna(axis=0)
#test = test.dropna(axis=0)

seed = 7 
seed2 = 77
tiney_fraction = 1e-05


In [2]:
train = train.dropna(subset=['GT_NO2'])
train.isnull().sum()

ID_Zindi                  0
Date                      0
ID                        0
LAT                       0
LON                       0
Precipitation             0
LST                   37594
AAI                   12118
CloudFraction         12118
NO2_strat             12118
NO2_total             12118
NO2_trop              33429
TropopausePressure    12118
GT_NO2                    0
dtype: int64

# Feature Engineering

In [3]:
groups = train['ID']
num_feats = train.select_dtypes(include=['float'])
kmeans = KMeans(n_clusters=2)

In [4]:
lat_min, lat_max = 44.92469405, 45.88973369
lon_min, lon_max = 8.736496578, 12.59068235

num_clusters_lat = 3
num_clusters_lon = 4
lat_step = (lat_max - lat_min) / num_clusters_lat
lon_step = (lon_max - lon_min) / num_clusters_lon
def assign_clusters(row, lat_step, lon_step, lat_min, lon_min):
    lat_cluster = int((row['LAT'] - lat_min) / lat_step)
    lon_cluster = int((row['LON'] - lon_min) / lon_step)
    return lat_cluster, lon_cluster
for dataset in (train, test):
    dataset[['lat_cluster', 'lon_cluster']] = dataset.apply(
        assign_clusters, axis=1, result_type='expand',
        lat_step=lat_step, lon_step=lon_step, lat_min=lat_min, lon_min=lon_min)

In [15]:
from khwarizm.time_series import time_features
"""
this is to get the time related features like year, month, week, day
"""
for df in (train,test):
    time_features(df)

train.columns

/workspaces/khwarizm/khwarizm/time_series.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset['date'] = pd.to_datetime(dataset[col])
/workspaces/khwarizm/khwarizm/time_series.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset['date'] = pd.to_datetime(dataset[col])


Index(['ID_Zindi', 'Date', 'ID', 'LAT', 'LON', 'Precipitation', 'LST', 'AAI',
       'CloudFraction', 'NO2_strat', 'NO2_total', 'NO2_trop',
       'TropopausePressure', 'GT_NO2', 'lat_cluster', 'lon_cluster', 'date',
       'Year', 'month', 'day', 'Weekday', 'Year_week', 'month_day'],
      dtype='object')

In [18]:
from khwarizm.time_series import rolling
rolling('NO2_trop', window=60)

ImportError: cannot import name 'rolling' from 'khwarizm.time_series' (/workspaces/khwarizm/khwarizm/time_series.py)

In [ ]:
stop

In [7]:
def rolling(feature):
    for dataset in (train,test):
        dataset[f"{feature}_rolling_max_60"] = dataset[feature].rolling(60).max()

rolling('NO2_trop')
rolling('NO2_total')
rolling('TropopausePressure')
rolling('CloudFraction')
rolling('Precipitation')

# Missing Values & Encoding

In [8]:
for df in(train,test):
    df.drop(columns=['Date','ID','Precipitation','CloudFraction','AAI','lat_cluster'], axis=1,inplace=True)

def encoder(train,test):
  le = LabelEncoder()
  for df in(train,test):
      for col in df.columns:
          if df[col].dtype == 'object':
              df[col] = le.fit_transform(df[col])
  return train,test

encoder(train,test)

(             LAT        LON     LST  NO2_strat  NO2_total  NO2_trop  \
 0      45.601585  11.903551     NaN   0.000024   0.000117       NaN   
 1      45.371005  11.840830     NaN   0.000024   0.000127       NaN   
 2      45.045825  12.060869     NaN   0.000024   0.000086       NaN   
 3      45.104075  11.553241     NaN   0.000024   0.000124       NaN   
 4      45.038758  11.790152     NaN   0.000024   0.000116       NaN   
 ...          ...        ...     ...        ...        ...       ...   
 86579  45.498227   9.556232     NaN   0.000032   0.000643       NaN   
 86580  45.142541  10.043836     NaN        NaN        NaN       NaN   
 86581  45.842207   9.351658  284.98   0.000031   0.000153  0.000122   
 86582  45.113503   8.874065     NaN   0.000031   0.000717       NaN   
 86583  45.526473   9.515980     NaN   0.000032   0.000643       NaN   
 
        TropopausePressure  GT_NO2  lon_cluster  Month  Month_Day  Year_Week  \
 0             14440.82126  31.000            3      1

# CV and Modeling

In [9]:
groups

0         PD01
1         PD04
2         RO01
3         RO02
4         RO03
         ...  
86579    X9897
86580    X9928
86581    X9969
86582    X9993
86583    X9999
Name: ID, Length: 82051, dtype: object

In [10]:

RandomForestModel = LGBMRegressor()
from khwarizm.feature_selection import validate_model
validate_model(model=RandomForestModel,cv='GroupKFold', n_splits=5,dataset=train,target_col='GT_NO2', groups=groups)

train shape : (65206,), test shape:(16845,)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013683 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2971
[LightGBM] [Info] Number of data points in the train set: 65206, number of used features: 18
[LightGBM] [Info] Start training from score 23.997347
std : 0        31.000
8        41.000
12       46.000
14       40.000
15       15.925
          ...  
86569    53.075
86572    36.325
86575    27.675
86580    30.125
86581    28.325
Name: GT_NO2, Length: 16845, dtype: float64
score : 10.457133906490098
train shape : (65234,), test shape:(16817,)


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004243 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2973
[LightGBM] [Info] Number of data points in the train set: 65234, number of used features: 18
[LightGBM] [Info] Start training from score 24.169432
std : 3        30.000
21       30.925
25       13.875
28       38.850
35       24.450
          ...  
86559    20.300
86573    27.300
86576    19.875
86577    24.225
86578    56.275
Name: GT_NO2, Length: 16817, dtype: float64
score : 10.664819793296964
train shape : (65834,), test shape:(16217,)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006648 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2970
[LightGBM] [Info] Number of data poi

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std : 1        42.000
4        58.000
6        38.000
10       40.000
22       35.400
          ...  
86551    44.750
86566    28.025
86574    29.225
86582    21.250
86583    40.350
Name: GT_NO2, Length: 16217, dtype: float64
score : 10.61840582290597
train shape : (65824,), test shape:(16227,)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004252 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2970
[LightGBM] [Info] Number of data points in the train set: 65824, number of used features: 18
[LightGBM] [Info] Start training from score 24.928707


...

std : 2        31.000
5        26.000
7        34.000
11       39.000
17       22.250
          ...  
86548    36.550
86552    52.625
86560    29.575
86564    25.175
86567    25.900
Name: GT_NO2, Length: 16227, dtype: float64
score : 9.036741834674645
train shape : (66106,), test shape:(15945,)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005790 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2974
[LightGBM] [Info] Number of data points in the train set: 66106, number of used features: 18
[LightGBM] [Info] Start training from score 25.277555


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std : 9        35.000
19       31.275
20       44.550
48       32.275
50       19.175
          ...  
86565    20.175
86568    33.875
86570    34.475
86571    34.500
86579    39.750
Name: GT_NO2, Length: 15945, dtype: float64
score : 9.645763216388659


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


np.float64(10.084572914751266)

In [11]:
"""model = CatBoostRegressor(
    iterations=1000,        
    learning_rate=0.1,      
    depth=6,                
    loss_function='RMSE',   
    eval_metric='RMSE',     
    random_seed=seed,          
    verbose=100,
   
)
"""

#model = CatBoostRegressor(random_state=seed)
#model = XGBRegressor(random_state= 7)
model =  LGBMRegressor(random_state=7)
n_splits = 5
n = train['GT_NO2'].count()
num_bins = int(1 + np.log2(n))
cv = GroupKFold(n_splits=n_splits)

def validate(trainset, testset, target_col):

    model.fit(trainset.drop(columns=target_col), trainset[target_col])
    pred = model.predict(testset.drop(columns=target_col))
    valid_idx = testset[target_col].notna()
    valid_testset = testset[target_col][valid_idx]
    valid_pred = pred[valid_idx]
    print('std:', valid_testset.std())
    score = mean_squared_error(valid_testset, valid_pred, squared=False)
    print('score:', score)

    return score
stds = []
rmse = []

for train_idx, test_idx in cv.split(train.drop(columns='GT_NO2'), train['GT_NO2'], groups=groups):
    train_v, test_v = train.iloc[train_idx], train.iloc[test_idx]
    stds.append(test_v['GT_NO2'].std())
    rmse.append(validate(train_v, test_v, 'GT_NO2'))

print('RMSE:', np.array(rmse).mean())
print('RMSE std:', np.array(rmse).std())
print('Standard Deviations:', stds)
print('RMSEs Deviations:', rmse)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007191 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2971
[LightGBM] [Info] Number of data points in the train set: 65206, number of used features: 18
[LightGBM] [Info] Start training from score 23.997347
std: 17.06148359415839
score: 10.457133906490098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006702 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2973
[LightGBM] [Info] Number of data points in the train set: 65234, number of used features: 18
[LightGBM] [Info] Start training from score 24.169432


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 19.215238775849844
score: 10.664819793296964
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005468 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2970
[LightGBM] [Info] Number of data points in the train set: 65834, number of used features: 18
[LightGBM] [Info] Start training from score 24.290743


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 18.587381576876247
score: 10.61840582290597
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005712 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2970
[LightGBM] [Info] Number of data points in the train set: 65824, number of used features: 18
[LightGBM] [Info] Start training from score 24.928707


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 14.468189480112546
score: 9.036741834674645


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


KeyboardInterrupt: 

In [61]:
def generate_specifications(model_name, num_folds, model_params, validation_type='StratifiedGroupKFold', shuffle=True):
    specs = []
    specs.append(f"Model: {model_name}")
    specs.append(f"Validation Type: {validation_type}")
    specs.append(f"Number of Folds: {num_folds}")
    specs.append(f"Shuffle: {'Yes' if shuffle else 'No'}")
    
    # Adding model parameters
    specs.append("Model Parameters:")
    for param, value in model_params.items():
        specs.append(f"  {param}: {value}")
    
    # Adding any additional info (if needed)
    specs.append("\nAdditional Notes:")
    specs.append("  -  as the evaluation metric.")
    
    # Joining all specifications into a single string
    return "\n".join(specs)

# Example usage

specifications = generate_specifications(model_name=str(model)[1:10], num_folds=n_splits, model_params=model.get_params())
print(specifications)

Model: GBMRegres
Validation Type: StratifiedGroupKFold
Number of Folds: 5
Shuffle: Yes
Model Parameters:
  boosting_type: gbdt
  class_weight: None
  colsample_bytree: 1.0
  importance_type: split
  learning_rate: 0.1
  max_depth: -1
  min_child_samples: 20
  min_child_weight: 0.001
  min_split_gain: 0.0
  n_estimators: 100
  n_jobs: None
  num_leaves: 31
  objective: None
  random_state: 7
  reg_alpha: 0.0
  reg_lambda: 0.0
  subsample: 1.0
  subsample_for_bin: 200000
  subsample_freq: 0

Additional Notes:
  -  as the evaluation metric.


# inference

In [63]:

model.fit(train.drop(columns='GT_NO2'),train['GT_NO2'])
y_pred = model.predict(test)
#sub_df = pd.DataFrame({'id': test_id,'GT_NO2':y_pred})
#sub_df.to_csv('submission9277409514290884.csv', index=False)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001309 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3012
[LightGBM] [Info] Number of data points in the train set: 48596, number of used features: 18
[LightGBM] [Info] Start training from score 22.909476


ValueError: Number of features of the model must match the input. Model n_features_ is 18 and input n_features is 14

In [ ]:
plt.figure(figsize=(16, 7))
sns.lineplot(x=train_date, y=train.GT_NO2, color='blue', label='Train')
sns.lineplot(x=test_date, y=y_pred, color='red', label='Prediction')
plt.legend()
plt.show()

In [ ]:
importances = model.feature_importances_
names = model.feature_names_
fi = pd.DataFrame({'Feature': names,
                   'importances': importances})
fi = fi.sort_values(by='importances', ascending=False)

fi.plot(kind='bar', x='Feature', y='importances', legend=False, figsize=(10, 6))
plt.title('Feature Importances')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.show()

In [ ]:
train.columns